**Approach 1.3:**

**Training:**

-> For every selected frame of video, extract features and give label(label generated by converting manually marked safe duration into frame intervals)

-> Use classification to predict 0/1 for each frame

**Testing :**

For every frame, extract features and use classification to predict label

**Features to be extracted from individual frames:**

Cummulative approach made after 1.2 with new improvement:
- Changed labels.csv file from manually annotated videos **second-wise** to **frame-wise**, frame-wise annotations generated using frame-wise-annotations.ipynb file.
- Added Relative speed of each vehicle as feature for all 6 regions (using saved speed arrays)

In [1]:
#importing necessary libraries
from __future__ import print_function
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
%matplotlib inline  
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, sqrt
import natsort
import pandas as pd
import random
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV 
import pickle

Using TensorFlow backend.


In [2]:
user = 'siddhi'

if user == 'siddhi':
    path_videos = 'C:/RoadCrossingAssistant/Data/Videos/'
    path_detection_arrays = 'C:/RoadCrossingAssistant/Data/Arrays_RetinaNet/'
    path_direction_arrays = 'C:/RoadCrossingAssistant/Data/Directions_RetinaNet/'
    path_labels_csv = 'C:/RoadCrossingAssistant/Data/labels_framewise_csv.csv'
    path_labels_list = 'C:/RoadCrossingAssistant/Data/labels_framewise_list.pkl'

elif user == 'yagnesh':
    path_videos = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/'
    path_detection_arrays = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays/arrays_v2_RetinaNet/'
    path_direction_arrays = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/directions/directions_v2_RetinaNet/'
    path_speed_arrays = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/speeds/'
    path_labels_csv = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.csv'
    path_labels_list = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.pkl'


detection_arrays = glob.glob(path_detection_arrays+'array*.npy')
detection_arrays = natsort.natsorted(detection_arrays)
#detection_arrays = np.array(detection_arrays)
#print(detection_arrays)
#print('shape of detection_arrays: ', detection_arrays.shape)

direction_arrays = glob.glob(path_direction_arrays+'directions*.npy')
direction_arrays = natsort.natsorted(direction_arrays)

speed_arrays = glob.glob(path_speed_arrays+'speeds*.npy')
speed_arrays = natsort.natsorted(speed_arrays)

videos = glob.glob(path_videos+'video*.MOV')
videos = natsort.natsorted(videos)
#videos = np.array(videos)
#print('shape of videos: ', videos.shape)

open_file = open(path_labels_list, "rb")
labels_list = pickle.load(open_file)
open_file.close()
#labels_list = np.array(labels_list)
#print('labels_array: \n ', labels_list[89])

In [14]:
#Perform train-test split(78-26)

x = np.arange(104)
#np.random.seed(42)
indices_test = np.random.choice(x, 24, replace=False)
indices_train = np.delete(x, indices_test, axis=0)

detection_arrays_train = [detection_arrays[ind] for ind in indices_train]
detection_arrays_test = [detection_arrays[ind] for ind in indices_test]

direction_arrays_train = [direction_arrays[ind] for ind in indices_train]
direction_arrays_test = [direction_arrays[ind] for ind in indices_test]

speed_arrays_train = [speed_arrays[ind] for ind in indices_train]
speed_arrays_test = [speed_arrays[ind] for ind in indices_test]

videos_train = [videos[ind] for ind in indices_train]
videos_test = [videos[ind] for ind in indices_test]

labels_train = [labels_list[ind] for ind in indices_train]
labels_test = [labels_list[ind] for ind in indices_test]

print('len of detection_arrays_train: ', len(detection_arrays_train))
print('len of detection_arrays_test: ', len(detection_arrays_test))
print('len of direction_arrays_train: ', len(direction_arrays_train))
print('len of direction_arrays_test: ', len(direction_arrays_test))
print('len of speed_arrays_train: ', len(speed_arrays_train))
print('len of speed_arrays_test: ', len(speed_arrays_test))
print('len of videos_train: ', len(videos_train))
print('len of videos_test: ', len(videos_test))
print('len of labels_train: ', len(labels_train))
print('len of labels_test: ', len(labels_test))


len of detection_arrays_train:  80
len of detection_arrays_test:  24
len of direction_arrays_train:  80
len of direction_arrays_test:  24
len of videos_train:  80
len of videos_test:  24
len of labels_train:  80
len of labels_test:  24


In [4]:
def find_region(x,y):

    '''
    Returns the region in which the particular point lies

    Parameters:
    x(int) : x coordinate of the point
    y(int) : y coordinate of the point

    Returns:
    int : region no (1/2/3/4/5/6)
    '''

    video_height = 1080
    video_width = 1920

    if(x<640 and y>=540):
        return 1 #Bottom left region
    if(x<=1280 and y>=540):
        return 2 #Bottom center region
    if(x>1280 and y>=540):
        return 3 #Bottom right region
    if(x>1280 and y<540):
        return 4 #Top right region
    if(x<=1280 and y<540):
        return 5 #Top center region
    if(x<640 and y<540):
        return 6 #Top left region

def extract_features_from_frame(bounding_boxes, directions_for_boxes, speed_for_boxes, frame_no):
        
    '''
    Extract features from a particular frame

    Parameters:
    bounding_boxes(array) : detection array of the video
    direction_for_boxes(array) : detection of direction for all bounding boxes 
    speed_for_boxes(array) : calculated speed arrays for all bounding boxes
    frame_no(int) : frame no for which features are to be extracted

    Returns:
    list : containing 24 features of the frame
    '''    

    no_of_boxes = len(bounding_boxes[frame_no])  # total number of bounding boxes
    boxes_frame = np.array(bounding_boxes[frame_no]) # bounding boxes for each frame 
    direction_frame = np.array(directions_for_boxes[frame_no]) # directions for every bounding boxes
    speed_frame = np.array(speed_for_boxes[frame_no]) # speed array for every bounding boxes

    # checking if no of bounding boxes and direction array length matches or not 
    if len(direction_frame) != no_of_boxes or len(speed_frame) != no_of_boxes:
        raise Exception('mismatch in number of boxes \n Direction Frame Length:'+ str(len(direction_frame))+'\n Speed Frame Length:' + str(len(speed_frame))+ '\n Total number of bounding boxes:' +str(no_of_boxes))

    # matching indices of bounding boxes with required directions array
    indices_required_boxes = np.where(direction_frame>=0)[0] # getting direction indices which is having value 0 or 1 (steady or positive direction vehicle)

    # assigning indices of interested direction values (0 and 1 value of directions)
    required_speed_frame = speed_frame[indices_required_boxes]
    boxes_required = boxes_frame[indices_required_boxes]

    # feature - number of vehicles in each region
    r1 = r2 = r3 = r4 = r5 = r6 = 0
    
    # feature - total area of vehicles in each region
    r1_area = r2_area = r3_area = r4_area = r5_area = r6_area = 0

    # distance calculation for every region
    bottom_center_point_x = 1920/2    # width/2
    bottom_center_point_y = 1080      # height
    #feature - minimum distance of the vechiles from every region to origin point (bottom-center point)
    r1_min_distance = r2_min_distance = r3_min_distance = r4_min_distance = r5_min_distance = r6_min_distance = 0

    # feature - Maximum speed of the vehicles from every region
    r1_max_speed = r2_max_speed = r3_max_speed = r4_max_speed = r5_max_speed = r6_max_speed = 0.0
    temp_speed = 0.0

    for j in range(len(boxes_required)):

        # center x coordinate of bounding box cx
        cx = (boxes_required[j][0] + boxes_required[j][2])/2

        # center y coordinate of bounding box cy
        cy = (boxes_required[j][1] + boxes_required[j][3])/2

        # area of bounding box
        bounding_box_area = ((boxes_required[j][3]-boxes_required[j][1])*
                        (boxes_required[j][2]-boxes_required[j][0]))

        # initial speed for frame
        temp_speed = required_speed_frame[j][0]

        # distance calculation for every bounding box
        distance = sqrt((cx - bottom_center_point_x)**2 + (cy - bottom_center_point_y)**2)

        # finding region according to cx,cy and finding total area, minimum distance and maximum speed for each region
        if(find_region(cx,cy)==1):
            r1 = r1 + 1
            r1_area = r1_area + bounding_box_area
            if(r1_min_distance==0 or distance < r1_min_distance):
                r1_min_distance = distance
            if(r1_max_speed > temp_speed):
                r1_max_speed = temp_speed
        elif(find_region(cx,cy)==2):
            r2 = r2 + 1
            r2_area = r2_area + bounding_box_area
            if(r2_min_distance==0 or distance < r2_min_distance):
                r2_min_distance = distance
            if(r2_max_speed > temp_speed):
                r2_max_speed = temp_speed
        elif(find_region(cx,cy)==3):
            r3 = r3 + 1
            r3_area = r3_area + bounding_box_area
            if(r3_min_distance==0 or distance < r3_min_distance):
                r3_min_distance = distance
            if(r3_max_speed > temp_speed):
                r3_max_speed = temp_speed
        elif(find_region(cx,cy)==4):
            r4 = r4 + 1
            r4_area = r4_area + bounding_box_area
            if(r4_min_distance==0 or distance < r4_min_distance):
                r4_min_distance = distance
            if(r4_max_speed > temp_speed):
                r4_max_speed = temp_speed
        elif(find_region(cx,cy)==5):
            r5 = r5 + 1
            r5_area = r5_area + bounding_box_area
            if(r5_min_distance==0 or distance < r5_min_distance):
                r5_min_distance = distance
            if(r5_max_speed > temp_speed):
                r5_max_speed = temp_speed
        elif(find_region(cx,cy)==6):
            r6 = r6 + 1
            r6_area = r6_area + bounding_box_area
            if(r6_min_distance==0 or distance < r6_min_distance):
                r6_min_distance = distance
            if(r6_max_speed > temp_speed):
                r6_max_speed = temp_speed
                
    features_frame = [r1,r1_area,r1_min_distance,r1_max_speed,r2,r2_area,r2_min_distance,r2_max_speed,r3,r3_area,r3_min_distance,r3_max_speed,r4,r4_area,r4_min_distance,r4_max_speed,r5,r5_area,r5_min_distance,r5_max_speed,r6,r6_area,r6_min_distance,r6_max_speed]  # final frame with 18 features
        
    return features_frame

In [5]:
def get_labels_from_video(no_frames, safe_duration_list):
    
    '''
    Get labels for a particular video 

    Parameters:
    no_frames(int) : no of frames in the given video
    safe_duration_list(list) : a list of the type [safe_start1, safe_end1, safe_start2, safe_end2,......]

    Returns:
    list : list with len = no of frames and the value at each index represents safe/unsafe at that frame_no (frame_no starting at 0)
    int : -1 if there is no safe duration in video, 1 otherwise
    '''

    labels = [0]*no_frames
    no_safe_durations = int(len(safe_duration_list)/2)
    if(no_safe_durations == 0):
        return labels,-1 # there is no safe duration in the given video so all labels marked 0
    else:

        for i in range(no_safe_durations):
            safe_start = max(safe_duration_list[i*2] - 1, 0)
            safe_end = min(safe_duration_list[i*2 +1] - 1, no_frames-1)
            labels[safe_start:safe_end+1] = [1]*(safe_end-safe_start+1) # marking the value b/w safe_start and safe_end with 1

    if len(labels) > no_frames: #len of labels cannot be greater than no_frames in video
        raise Exception('Check the labels assigned in CSV file!')
    return labels,1

In [28]:
def generate_dataframe(indices):

    '''
    Generate dataframes for model training

    Parameters:
    indices(list) : indices_train for generating training dataframes, indices_test for generating testing dataframes

    Returns:
    features_dataframe : dataframe containg 24 features per frame
    labels_dataframe : dataframe containing 1 label per frame
    '''


    cols_f = ['region1','region1_area','region1_min_distance','region1_max_speed','region2','region2_area','region2_min_distance','region2_max_speed','region3','region3_area','region3_min_distance','region3_max_speed','region4','region4_area','region4_min_distance','region4_max_speed','region5','region5_area','region5_min_distance','region5_max_speed','region6','region6_area','region6_min_distance','region6_max_speed'] # 24 features per frame

    cols_l = ['safe/unsafe'] #1 label per frame
    features_dataframe = pd.DataFrame()
    labels_dataframe = pd.DataFrame()

    for ind in indices:

        fname = detection_arrays[ind]
        dname = direction_arrays[ind]
        sname = speed_arrays[ind]
        safe_duration_list = labels_list[ind]

        print("processing ",fname)
        bounding_boxes = np.load(fname, allow_pickle=True) #loading the numpy array containing all detected vehicles
        directions_for_boxes = np.load(dname, allow_pickle=True) # loading the arrays containing detected directions for each bounding box
        speed_for_boxes = np.load(sname, allow_pickle=True) # loading the arrays containing calculated speed for each bounding box

        no_frames = bounding_boxes.shape[0]
        #print("no of frames: ", no_frames)
        labels, flag = get_labels_from_video(no_frames, safe_duration_list)
        #print('flag', flag)

        labels_0 = [i for i, value in enumerate(labels) if value == 0 and i>6]
        # shuffling unsafe frames
        random.shuffle(labels_0)
        # selecting 50 frames from unsafe videos instead of rejecting the whole video
        if(len(labels_0) >= 50): 
            labels_0 = labels_0[:50]

        if(flag == -1):
            #print('no safe time')
            labels_1 = []
        else:
            labels_1 = [i for i, value in enumerate(labels) if value == 1 and i>6]
            # shuffling safe frames
            random.shuffle(labels_1)
            ## selecting no of safe frames = no of unsafe frames from each video ##
            # label 0 frames will be more than label 1 frame, 
            # so we will try to select same number of safe and unsafe frames
            if(len(labels_0) >= len(labels_1)):
                labels_0 = labels_0[:len(labels_1)]

        features = []
 

        for frame_no in labels_0:
            features_frame = extract_features_from_frame(bounding_boxes, directions_for_boxes, speed_for_boxes, frame_no)
            features.append(features_frame)
        for frame_no in labels_1:
            features_frame = extract_features_from_frame(bounding_boxes, directions_for_boxes, speed_for_boxes, frame_no)
            features.append(features_frame)
        
        # generating and returning features and labels dataframes
        no_frames_in_features = len(features)
        #print("no_frames_in_features",no_frames_in_features)
        labels = [0]*len(labels_0) + [1]*len(labels_1)
        #print("no_of_frames_in_labels", len(labels))
        df1 = pd.DataFrame(features, columns=cols_f)
        features_dataframe = features_dataframe.append(df1,ignore_index=True)
        print("df1.size",df1.shape)
        df2 = pd.DataFrame(labels, columns=cols_l)
        labels_dataframe = labels_dataframe.append(df2,ignore_index=True)
        #print("df2.size",df2.shape)
        
    return features_dataframe, labels_dataframe

In [15]:
#modified function in terms of selection of frames

def generate_dataframe(indices, reduce):

    '''
    Generate dataframes for model training

    Parameters:
    indices(list) : indices_train for generating training dataframes, indices_test for generating testing dataframes
    reduce(bool) : True to reduce dataframe size to maintain safe/unsafe ratio, else False

    Returns:
    features_dataframe : dataframe containg 24 features per frame
    labels_dataframe : dataframe containing 1 label per frame
    '''


    cols_f = ['region1','region1_area','region1_min_distance','region1_max_speed','region2','region2_area','region2_min_distance','region2_max_speed','region3','region3_area','region3_min_distance','region3_max_speed','region4','region4_area','region4_min_distance','region4_max_speed','region5','region5_area','region5_min_distance','region5_max_speed','region6','region6_area','region6_min_distance','region6_max_speed'] # 24 features per frame

    cols_l = ['safe/unsafe'] # 1 label per frame
    features_dataframe = pd.DataFrame()
    labels_dataframe = pd.DataFrame()

    for ind in indices:

        fname = detection_arrays[ind]
        dname = direction_arrays[ind]
        sname = speed_arrays[ind]
        safe_duration_list = labels_list[ind]

        print("processing ",fname)
        bounding_boxes = np.load(fname, allow_pickle=True) # loading the numpy array containing all detected vehicles
        directions_for_boxes = np.load(dname, allow_pickle=True) # loading the arrays containing detected directions for each bounding box
        speed_for_boxes = np.load(sname, allow_pickle=True) # loading the arrays containing speed for each bounding box

        no_frames = bounding_boxes.shape[0]
        #print("no of frames: ", no_frames)
        labels, flag = get_labels_from_video(no_frames, safe_duration_list)
        labels = labels[6:]
        #print('flag', flag)

        #labels_0 = [i for i, value in enumerate(labels) if value == 0]
        # shuffling unsafe frames
        #random.shuffle(labels_0)
        # selecting 50 frames from unsafe videos instead of rejecting the whole video
        #if(len(labels_0) >= 50): 
        #    labels_0 = labels_0[:50]

        #if(flag == -1):
        #    #print('no safe time')
        #    labels_1 = []
        # else:
        #     labels_1 = [i for i, value in enumerate(labels) if value == 1]
        #     # shuffling safe frames
        #     random.shuffle(labels_1)
        #     ## selecting no of safe frames = no of unsafe frames from each video ##
        #     # label 0 frames will be more than label 1 frame, 
        #     # so we will try to select same number of safe and unsafe frames
        #     if(len(labels_0) >= len(labels_1)):
        #         labels_0 = labels_0[:len(labels_1)]

        features = []
 
        for frame_no in range(6, no_frames):
            features_frame = extract_features_from_frame(bounding_boxes, directions_for_boxes, speed_for_boxes,frame_no)
            features.append(features_frame)
        
        #generating and returning features and labels dataframes
        no_frames_in_features = len(features)
        #print("no_frames_in_features",no_frames_in_features)
        #labels = [0]*len(labels_0) + [1]*len(labels_1)
        #print("no_of_frames_in_labels", len(labels))
        df1 = pd.DataFrame(features, columns=cols_f)
        features_dataframe = features_dataframe.append(df1,ignore_index=True)
        #print("df1.size",df1.shape)
        df2 = pd.DataFrame(labels, columns=cols_l)
        labels_dataframe = labels_dataframe.append(df2,ignore_index=True)
        #print("df2.size",df2.shape)
    
    if reduce == True:
        ind0 = labels_dataframe.index[labels_dataframe['safe/unsafe'] == 0].tolist()
        random.shuffle(ind0)
        ind1 = labels_dataframe.index[labels_dataframe['safe/unsafe'] == 1].tolist()
        random.shuffle(ind1)
        print(len(ind0)/len(ind1))
        if (len(ind0)/len(ind1)) > 1.2:
            print('reducing the number of unsafe frames in dataframe\n\n')
            len_ind0 = int(len(ind1)*1.2)
            ind0 = ind0[:len_ind0]

            indices_required = ind0 + ind1
            features_dataframe = features_dataframe.iloc[indices_required]
            labels_dataframe = labels_dataframe.iloc[indices_required]
        
    return features_dataframe, labels_dataframe

In [16]:
features_df_train, labels_df_train = generate_dataframe(indices_train, True)
# features_df_train.to_pickle('features_df_train.pkl')
# labels_df_train.to_pickle('labels_df_train.pkl')

features_df_test, labels_df_test = generate_dataframe(indices_test, False)
# features_df_test.to_pickle('features_df_test.pkl')
# labels_df_test.to_pickle('labels_df_test.pkl')
print("features_df_train.shape: ", features_df_train.shape)
print("labels_df_train.shape: ", labels_df_train.shape)

print("features_df_test.shape: ", features_df_test.shape)
print("labels_df_test.shape: ", labels_df_test.shape)

processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array1.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array2.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array3.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array4.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array5.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array6.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array7.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array8.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array9.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array10.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array11.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array12.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array13.npy
processing  C:/RoadCrossingAssistant/Data/Arrays_RetinaNet\array14.npy
processing  C:/

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# StandardScaler
#feature_scaler = StandardScaler()

# MinMaxScaler
feature_scaler = MinMaxScaler()

# storing scaled features
features_train_scaled = pd.DataFrame(feature_scaler.fit_transform(features_df_train))
features_train_scaled.columns = ['region1','region1_area','region1_min_distance','region1_max_speed','region2','region2_area','region2_min_distance','region2_max_speed','region3','region3_area','region3_min_distance','region3_max_speed','region4','region4_area','region4_min_distance','region4_max_speed','region5','region5_area','region5_min_distance','region5_max_speed','region6','region6_area','region6_min_distance','region6_max_speed']
features_test_scaled = pd.DataFrame(feature_scaler.transform(features_df_test))
features_test_scaled.columns = ['region1','region1_area','region1_min_distance','region1_max_speed','region2','region2_area','region2_min_distance','region2_max_speed','region3','region3_area','region3_min_distance','region3_max_speed','region4','region4_area','region4_min_distance','region4_max_speed','region5','region5_area','region5_min_distance','region5_max_speed','region6','region6_area','region6_min_distance','region6_max_speed']

## Model Training
### Classification model: Support Vector Machine with rbf kernel

In [18]:
# scaled dataset
model = SVC()
model.fit(features_train_scaled, labels_df_train) 
from sklearn.metrics import accuracy_score
 
# print prediction results 
print("for train data:")
predictions = model.predict(features_train_scaled) 
print(classification_report(labels_df_train, predictions)) 
print("accuracy score", accuracy_score(labels_df_train, predictions))

print("\nfor test data:")
predictions = model.predict(features_test_scaled) 
print(classification_report(labels_df_test, predictions)) 
print("accuracy score",accuracy_score(labels_df_test, predictions))


for train data:
              precision    recall  f1-score   support

           0       0.84      0.79      0.81      8443
           1       0.77      0.82      0.79      7036

    accuracy                           0.80     15479
   macro avg       0.80      0.80      0.80     15479
weighted avg       0.81      0.80      0.80     15479

accuracy score 0.8028942438141999

for test data:
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      3871
           1       0.67      0.63      0.65      1925

    accuracy                           0.77      5796
   macro avg       0.74      0.73      0.74      5796
weighted avg       0.77      0.77      0.77      5796

accuracy score 0.7715665976535542


In [21]:
# checking predictions for individual frames

def prediction_for_frame(aname, dname, frame_no ):

    bounding_boxes = np.load(aname, allow_pickle=True)
    dirs = np.load(dname, allow_pickle=True)

    features = extract_features_from_frame(bounding_boxes, dirs, frame_no)

    f = []
    f.append(features)
    cols_f = ['region1','region1_area','region1_min_distance','region1_max_speed','region2','region2_area','region2_min_distance','region2_max_speed','region3','region3_area','region3_min_distance','region3_max_speed','region4','region4_area','region4_min_distance','region4_max_speed','region5','region5_area','region5_min_distance','region5_max_speed','region6','region6_area','region6_min_distance','region6_max_speed'] # 24 features per frame
    f_df = pd.DataFrame(f, columns=cols_f)

    f_df_scaled = pd.DataFrame(feature_scaler.transform(f_df))
    f_df_scaled.columns = cols_f

    print("prediction: ",model.predict(f_df_scaled))

def show_frame(vname, frame_no): 
      
    # Path to video file 
    vidObj = cv2.VideoCapture(vname) 
    # Used as counter variable 
    count = 0
    # checks whether frames were extracted 
    success = 1
  
    while success: 
  
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read() 
        if(count == frame_no):
        # Saves the frames with frame-count 
            cv2.imwrite("frame%d.jpg" % count, image)
            break 
  
        count += 1


aname = path_detection_arrays + "/array62.npy"
vname = "/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/video62.MOV"
dname = path_direction_test_arrays + "/directions62.npy"
frame_no  = 160

prediction_for_frame(aname, dname, frame_no)
show_frame(vname, frame_no)


NameError: name 'path_direction_test_arrays' is not defined